## 套件匯入

In [1]:
from selenium import webdriver
from time import sleep

## 啟動瀏覽器（https://law.judicial.gov.tw/FJUD/default.aspx )

In [2]:
# 司法院官網
link = 'https://law.judicial.gov.tw/FJUD/default.aspx'
options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.get(link)

# 定位輸入框，寫入搜尋 <關鍵字>
TEXT = '證券詐欺'
sleep(2)
driver.find_element_by_id('txtKW').send_keys(TEXT)
driver.find_element_by_id('btnSimpleQry').click()

# 等待載入，選擇最高法院（若有）
try:
    driver.find_element_by_xpath("//div[@class='panel-body']   \
    //li/a[contains(text(),'最高法院')]").click()
except NoSuchElementException:
    pass

## 網頁檢索 / 爬蟲

In [3]:
# 建立須被彈性設定之變數
TIME = '109'
data = list()
status = True

# 由於定位資料於 iframe 內
# 因此需要告知瀏覽器切換到 iframe 嵌入框才能進行定位
iframe = driver.find_element_by_xpath("//iframe")
driver.switch_to.frame(iframe)

while(status):
    # 定位該頁所有搜尋結果之資訊，並進行迴圈檢視
    for position,each_date in enumerate(driver.find_elements_by_xpath("//table//tr[position() > 1 and not(contains(@class,'summary'))]/td[3]")):
        if each_date.text[:len(TIME)] == TIME:
            xpath_to_data = "(//table//tr[position() > 1 and not(contains(@class,'summary'))]/td[{}])[position() = {}]"
            data.append({
                'title'   : driver.find_element_by_xpath(xpath_to_data.format(2,position+1)).text,
                'date'    : driver.find_element_by_xpath(xpath_to_data.format(3,position+1)).text,
                'judge'   : driver.find_element_by_xpath(xpath_to_data.format(4,position+1)).text,
                'content' : '',
                'link'    : driver.find_element_by_xpath(f'{xpath_to_data.format(2,position+1)}//a').get_attribute("href")
            })  
    next_ = driver.find_elements_by_id('hlNext')
    if len(next_) != 0:
        next_[0].click()
    else:
        status = False

## 指定網頁判決內容截取

In [7]:
for each in data:
    driver = webdriver.Chrome(options=options)
    driver.get(each['link'])
    each['content'] = driver.find_element_by_xpath("(//div[@class='text-pre text-pre-in'])").text
    driver.close()
    sleep(1)

## 資料輸出

In [8]:
data

[{'title': '最高法院 109 年度 台上 字第 1719 號刑事判決（29K）',
  'date': '109.04.15',
  'judge': '違反證券交易法等罪',
  'content': '最高法院刑事判決\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000 108年度台上字第4056號\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000 109年度台上字第1719號\n上\u3000訴\u3000人\u3000臺灣高等檢察署檢察官黃騰耀\n上\u3000訴\u3000人\u3000\n（被\u3000告）\u3000王宗立\n選任辯護人\u3000張簡勵如律師\n\u3000\u3000\u3000\u3000\u3000\u3000林宗憲律師\n上\u3000訴\u3000人\u3000王安石（原名王宗貞）\n（被\u3000告）\u3000\u3000\u3000\u3000\n原審辯護人\u3000王寶蒞律師\n上\u3000訴\u3000人\u3000\n（被\u3000告）\u3000劉玉增\n選任辯護人\u3000丁中原律師\n\u3000\u3000\u3000\u3000\u3000\u3000林俊吉律師\n被\u3000\u3000\u3000告\u3000楊善博（原名楊偉祥）\n\u3000\u3000\u3000\u3000\u3000\u3000林憶慧（原名林月桂）\n選任辯護人\u3000王福民律師\n上列上訴人等因被告等違反證券交易法等罪案件，不服臺灣高等\n法院中華民國108年8月1日第二審判決（106年度金上重訴字第11\n號、106 年度上訴字第1052號，起訴案號：臺灣臺北地方檢察署\n90年度偵字第21111、23982 號，91年度偵字第9844、10256、25\n181號，92年度偵字第1501、20496、24888號，93年度偵字第500\n、10184、10344、13340號，94年度偵字第840、841、842、7661\n、14564 號；追加起訴案號：105年度蒞追字第4